In [1]:
import json
import os


import rootpath
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib
# matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import geopandas as gpd
import pycountry as pc
import pycountry_convert as pcc
import powerlaw
from matplotlib.patches import Circle
from networkx.drawing.nx_agraph import graphviz_layout
from sklearn.linear_model import LinearRegression

from lhledge import cfgLoader
from lhledge import lhlFilters
from lhledge import loadGeographicData

In [2]:
CYCLE = 8820
DATE = 20201002
# CYCLE = 4578
# DATE = 20160302
DOWNSAMPLING = 1

In [3]:
def unfold_prefix(prefix):
    net, mask = prefix.split("/")

    o0, o1, o2, o3 = np.array(net.split(".")).astype(int)

    if int(mask) == 24:
        # return net
        return [net,]
    elif int(mask) < 24:
        p = []
        for i in range(24 - int(mask)):
            p.append(f"{o0}.{o1}.{o2+ i}.{o3}")
        return p
    else:
        return [net,]



In [4]:
def toslash24(prefix):

    o0, o1, o2, o3 = np.array(prefix.split(".")).astype(int)

    return f"{o0}.{o1}.{o2}.0"

In [5]:
def ecdf(data, w=[]):
    """ Compute ECDF """
    if len(w) == 0:
        w = np.ones(len(data))
    #
    #     x = np.sort(data)
    idx = np.argsort(data)
    #
    x = np.array(data)
    x = x[idx]
    w = w[idx]
    #
    n = x.size
    #     y = np.arange(1, n + 1) / n
    y = np.cumsum(w) / sum(w)
    return (np.squeeze(x), y)

In [6]:
def geneate_input_filename(date, cycle):

    filename = cfg["paths"]["ark"]["consolidated-stable-long-haul-explorations"].format(date, cycle, DOWNSAMPLING)

    # filenames._create_dir(filename)
    return filename

In [9]:
# Change directory to run from the root dir of the project
path = rootpath.detect(pattern=".git")
os.chdir(path)

# load config file
cfg = cfgLoader.cfgLoader("config.yml")

In [10]:
def _get_continent(cc):
    if cc == "":
        return ""

    try:
        country = pc.countries.get(alpha_2=cc)
        continent = pcc.country_alpha2_to_continent_code(country.alpha_2)
        return continent
    except:
        return ""

In [11]:
def filter_with_cepii(cepii, geoloc_hops):

    # agrego CEPII y elimino cortos

    SoL_TH = 100
    geoloc_hops["near_side_cont"] = geoloc_hops["near_side_cc"].map(_get_continent)
    geoloc_hops["far_side_cont"] = geoloc_hops["far_side_cc"].map(_get_continent)

    tmp = geoloc_hops.loc[(geoloc_hops["diff_rtt"] > 57) \
                    & (geoloc_hops["near_side_cont"]
                       != geoloc_hops["far_side_cont"])] \
            [["near_side_addr", "far_side_addr",
              "near_side_cc", "far_side_cc",
              "near_node_id", "far_node_id",
              "near_node_asn", "far_node_asn",
              "near_side_lat", "far_side_lat",
              "near_side_lon", "far_side_lon",
              "diff_rtt"]]

    tmp = tmp.groupby(["near_side_cc", "far_side_cc",
                       "near_side_addr", "far_side_addr",
                       "near_node_id", "far_node_id",
                       "near_side_lat", "far_side_lat",
                       "near_side_lon", "far_side_lon",
                       "near_node_asn", "far_node_asn",]) \
        .min()["diff_rtt"] \
        .reset_index()

    tmp = tmp.join(
        cepii[["cc_src", "cc_dst", "dist"]].set_index(["cc_src", "cc_dst"]),
        on=["near_side_cc", "far_side_cc"],
        how='left',
        lsuffix='_left',
        rsuffix='_right'
    )

    # Cond 1: no violation of speed of light contrains
    # Cond 2: No long detours. Latency can't be twice the inter-country distance
    tmp = tmp.loc[(tmp["diff_rtt"] > (tmp["dist"] / SoL_TH))
                   & ((tmp["diff_rtt"]  * SoL_TH) < (2 * tmp["dist"]))]
    return tmp

In [12]:
def ixp_flag(x):
    if len(x) == 0:
        return False

    return True

def add_ixp_data(lhl, ixps_nets):


    lhl["near_prefix"] =  lhl["near_side_addr"].map(toslash24)
    lhl["far_prefix"] =  lhl["far_side_addr"].map(toslash24)

    lhl = lhl.join(
        ixps_nets.set_index(["net", ]),
        on=["near_prefix", ],
        how='left',
        lsuffix='_left',
        rsuffix='_near'
    )
    lhl = lhl.rename(columns = {'ixp': 'near_side_ixp'})

    lhl = lhl.join(
        ixps_nets.set_index(["net", ]),
        on=["far_prefix", ],
        how='left',
        lsuffix='_left',
        rsuffix='_far'
    )
    lhl = lhl.rename(columns = {'ixp': 'far_side_ixp'})

    lhl['near_side_ixp'] = lhl['near_side_ixp'].fillna("")
    lhl['far_side_ixp'] = lhl['far_side_ixp'].fillna("")

    lhl["flag_near_side_ixp"] = lhl["near_side_ixp"].map(ixp_flag)
    lhl["flag_far_side_ixp"] = lhl["far_side_ixp"].map(ixp_flag)

    return lhl

# Load data

## IXP data

In [13]:
with open("data/raw/ixps/ixs_202110.json") as json_file:
    ixps = json.load(json_file)

In [14]:
ixps_nets = []

for ixp in ixps:
    for prefix in ixp["prefixes"]["ipv4"]:
        for slash24 in unfold_prefix(prefix):
            ixps_nets.append((ixp["name"], slash24))

ixps_nets = pd.DataFrame(ixps_nets, columns=["ixp", "net"])
ixps_nets.head()


,ixp,net
0,1-IX Internet Exchange,185.1.213.0
1,36C3-YOLOIXP,185.236.243.0
2,48 IX,149.112.3.0
3,4b42 Internet Exchange Point,185.1.125.0
4,6NGIX,203.254.32.0


## Geographic data

In [16]:
cepii = loadGeographicData.load_inter_country_distances("data/external/cepii/dist_cepii.csv", 
                                     "data/processed/min-cc-dist/min_cc_dist.csv")
cepii.head()

,cc_src,cc_dst,dist
0,AW,AW,5.225315
1,AW,AF,13257.810000
2,AW,AO,9516.913000
3,AW,AI,983.268200
4,AW,AL,9091.742000


## LHLs

In [19]:
filtered_geoloc_hops = {}

for date, cycle in [(20220501, 10019), (20220501, 10020), (20220502, 10021),
                    (20211001, 9643), (20211001, 9644), (20211002, 9645),
                    (20201009, 8838), (20201009, 8839), (20201009, 8840),
                    (20201002, 8820), (20201002, 8821), (20201002, 8822),
                    (20160302, 4576), (20160302, 4577), (20160302, 4578),
                    (20190801, 7615), (20190801, 7616), (20190801, 7617),
                    (20180301, 6446), (20180301, 6447), (20180301, 6448),
                    (20170201, 5422), (20170201, 5423), (20170201, 5424),]:   

    lhl = pd.read_csv(geneate_input_filename(date, cycle),
                               compression="bz2")
    lhl["mpls_tunnel"] = lhl["mpls_tunnel"].astype(bool)

    _geoloc_hops = lhl.drop_duplicates(["near_side_addr", "far_side_addr"]) \
                      .loc[(lhl["near_side_cc"] != "??")
                           & (lhl["far_side_cc"] != "??")
                           & (lhl["far_side_rtt"] >  lhl["near_side_rtt"])] \
                      [["near_side_addr", "far_side_addr",
                        "near_node_id", "far_node_id",
                        "near_side_cc", "far_side_cc",
                        "near_side_lat", "far_side_lat",
                        "near_side_lon", "far_side_lon",
                        "near_node_asn", "far_node_asn",]]

    min_near = lhl.groupby(["near_side_addr", "far_side_addr"])["near_side_rtt"] \
        .min() \
        .reset_index()

    min_far = lhl.groupby(["near_side_addr", "far_side_addr"])["far_side_rtt"] \
        .min() \
        .reset_index()


    _geoloc_hops = _geoloc_hops.join(
            min_near.set_index(["near_side_addr", "far_side_addr"]),
            on=["near_side_addr", "far_side_addr"],
            how='left',
            lsuffix='_left',
            rsuffix='_right'
        )

    _geoloc_hops = _geoloc_hops.join(
            min_far.set_index(["near_side_addr", "far_side_addr"]),
            on=["near_side_addr", "far_side_addr"],
            how='left',
            lsuffix='_left',
            rsuffix='_right'
        )


    _geoloc_hops["diff_rtt"] = _geoloc_hops["far_side_rtt"] - _geoloc_hops["near_side_rtt"]

    _geoloc_hops = _geoloc_hops.loc[(_geoloc_hops["near_side_cc"].notnull())
                                    & (_geoloc_hops["far_side_cc"].notnull())]


    filtered_geoloc_hops[cycle] = filter_with_cepii(cepii, _geoloc_hops)
    filtered_geoloc_hops[cycle] = add_ixp_data(filtered_geoloc_hops[cycle], ixps_nets)


    

In [21]:
asrel_mapping ={
    4577: 20160301,
    5423: 20170201,
    6447: 20180401,
    7616: 20190801,
    8821: 20201001,
    9644: 20211001,
    10020: 20220501
}



# for cycle in [4577, 5423, 6447, 7616, 8821, 8839]:
for cycle in [4577, 5423, 6447, 7616, 8821, 9644, 10020]:
    concat_df = pd.DataFrame()
    for _cycle in range(cycle -1, cycle + 2):
        concat_df = pd.concat([concat_df, filtered_geoloc_hops[_cycle]])

    G = nx.from_pandas_edgelist(
        concat_df.loc[concat_df["diff_rtt"] > 40],
        "near_node_id",
        "far_node_id",
    )

    nx.set_node_attributes(
        G,
        pd.Series(
            concat_df["near_node_asn"].values,
            index=concat_df["near_node_id"]
        ).to_dict(),
        'asn',
    )
    nx.set_node_attributes(
        G,
        pd.Series(
            concat_df["far_node_asn"].values,
            index=concat_df["far_node_id"]
        ).to_dict(),
        'asn',
    )

    asrel = pd.read_csv(
        f"data/raw/asrel/{asrel_mapping[cycle]}.as-rel.txt.bz2",
        compression="bz2",
        sep="|",
        comment="#",
        names=["provider", "customer", "type"]
    )

    t = len(G.edges())

    i = 0
    p2p = 0
    p2c = 0
    tor_uknown = 0
    unmapped = 0

    for node in G.nodes():
        for neighbor_id in G.neighbors(node):

            near_asn = int(G.nodes[node]["asn"])
            far_asn = int(G.nodes[neighbor_id]["asn"])

            if (near_asn != 0) and (near_asn != 0):
                if near_asn == far_asn:
                    i += 1
                else:
                    tor = asrel.loc[
                        ((asrel["provider"] == near_asn)
                         & (asrel["customer"] == far_asn))
                        | ((asrel["customer"] == near_asn)
                           & (asrel["provider"] == far_asn))
                    ]["type"].values

                    if len(tor) > 0:
                        tor = tor[0]
                    else:
                        tor = 1

                    if tor == -1:
                        p2c +=1
                    elif tor == 0:
                        p2p += 1
                    else:
                        tor_uknown += 1
            else:
                unmapped += 1
    t *= 2
    print(f"{cycle}\t{i}\t{p2p}\t{p2c}\t{tor_uknown}\t{unmapped}\t{t}\t{(i / float(t)):.02f}\t{(p2p / float(t)):.02f}\t{(p2c / float(t)):.02f}\t{(tor_uknown / float(t)):.02f}")

4577	10202	858	1714	661	17	13452	0.76	0.06	0.13	0.05
5423	12346	624	2012	1190	40	16212	0.76	0.04	0.12	0.07
6447	14172	864	2312	1192	0	18540	0.76	0.05	0.12	0.06
7616	24812	1622	3588	1276	224	31522	0.79	0.05	0.11	0.04
8821	29264	718	2556	740	2	33280	0.88	0.02	0.08	0.02
9644	41354	1278	6138	1532	2	50304	0.82	0.03	0.12	0.03
10020	36796	870	5140	1694	0	44500	0.83	0.02	0.12	0.04


In [22]:
len(G.edges())

22250

In [23]:
for cycle in [10020, ]:
    concat_df = pd.DataFrame()
    for _cycle in range(cycle -1, cycle + 2):
        # concat_df = concat_df.append(filtered_geoloc_hops[_cycle])
        concat_df = pd.concat([concat_df, filtered_geoloc_hops[_cycle]])

G = nx.from_pandas_edgelist(
    concat_df.loc[concat_df["diff_rtt"] > 40],
    "near_node_id",
    "far_node_id",
)

In [24]:
concat_df.shape

(46224, 20)

In [25]:
asrel_mapping ={
    4577: 20160301,
    5423: 20170201,
    6447: 20180401,
    7616: 20190801,
    8821: 20201001,
    9644: 20211001,
    10020: 20220501
}

asrel = pd.DataFrame()

for cycle in asrel_mapping.keys():
    _asrel = pd.read_csv(
        f"data/raw/asrel/{asrel_mapping[cycle]}.as-rel.txt.bz2",
        compression="bz2",
        sep="|",
        comment="#",
        names=["provider", "customer", "type"]
    )
    # asrel = asrel.append(_asrel)
    asrel = pd.concat([asrel, _asrel])


for cycle in [4577, 5423, 6447, 7616, 8821, 9644, 10020]:
    concat_df = pd.DataFrame()
    for _cycle in range(cycle -1, cycle + 2):
        # concat_df = concat_df.append(filtered_geoloc_hops[_cycle])
        concat_df = pd.concat([concat_df, filtered_geoloc_hops[_cycle]])

    G = nx.from_pandas_edgelist(
        concat_df.loc[concat_df["diff_rtt"] > 40],
        "near_node_id",
        "far_node_id",
    )

    nx.set_node_attributes(
        G,
        pd.Series(
            concat_df["near_node_asn"].values,
            index=concat_df["near_node_id"]
        ).to_dict(),
        'asn',
    )
    nx.set_node_attributes(
        G,
        pd.Series(
            concat_df["far_node_asn"].values,
            index=concat_df["far_node_id"]
        ).to_dict(),
        'asn',
    )


    nx.set_node_attributes(
        G,
        pd.Series(
            concat_df["flag_far_side_ixp"].values,
            index=concat_df["near_node_id"]
        ).to_dict(),
        'ixp_flag',
    )
    nx.set_node_attributes(
        G,
        pd.Series(
            concat_df["flag_near_side_ixp"].values,
            index=concat_df["far_node_id"]
        ).to_dict(),
        'ixp_flag',
    )

    t = len(G.edges())

    i = 0
    p2p = 0
    p2c = 0
    ixp = 0
    tor_uknown = 0
    unmapped = 0

    for node in G.nodes():
        for neighbor_id in G.neighbors(node):

            near_asn = int(G.nodes[node]["asn"])
            far_asn = int(G.nodes[neighbor_id]["asn"])

            if (near_asn != 0) and (near_asn != 0):
                if near_asn == far_asn:
                    i += 1
                else:
                    tor = asrel.loc[
                        ((asrel["provider"] == near_asn)
                         & (asrel["customer"] == far_asn))
                        | ((asrel["customer"] == near_asn)
                           & (asrel["provider"] == far_asn))
                    ]["type"].values

                    if len(tor) > 0:
                        tor = tor[0]
                    else:
                        tor = 1

                    if tor == -1:
                        p2c +=1
                    elif tor == 0:
                        p2p += 1
                    else:
                        if G.nodes[node]["ixp_flag"] or G.nodes[neighbor_id]["ixp_flag"]:
                            ixp +=1
                        else:
                            tor_uknown += 1
            else:
                unmapped += 1
    i /= 2
    p2p /= 2
    p2c /= 2
    tor_uknown /= 2

    print(f"{cycle}\
    &{i} ({(i / float(t)):.02f})\
    &{p2p} ({(p2p / float(t)):.02f})\
    &{p2c} ({(p2c / float(t)):.02f})\
    &{ixp} ({(ixp / float(t)):.02f})\
    &{tor_uknown} ({(tor_uknown / float(t)):.02f})\
    &{unmapped} ({(unmapped / float(t)):.02f})\
    &{t}")


4577    &5101.0 (0.76)    &461.0 (0.07)    &893.0 (0.13)    &131 (0.02)    &197.0 (0.03)    &17 (0.00)    &6726
5423    &6173.0 (0.76)    &494.0 (0.06)    &1044.0 (0.13)    &130 (0.02)    &310.0 (0.04)    &40 (0.00)    &8106
6447    &7086.0 (0.76)    &602.0 (0.06)    &1286.0 (0.14)    &170 (0.02)    &211.0 (0.02)    &0 (0.00)    &9270
7616    &12406.0 (0.79)    &873.0 (0.06)    &1873.0 (0.12)    &187 (0.01)    &403.5 (0.03)    &224 (0.01)    &15761
8821    &14632.0 (0.88)    &465.0 (0.03)    &1285.0 (0.08)    &90 (0.01)    &212.0 (0.01)    &2 (0.00)    &16640
9644    &20677.0 (0.82)    &915.0 (0.04)    &2961.0 (0.12)    &100 (0.00)    &548.0 (0.02)    &2 (0.00)    &25152
10020    &18398.0 (0.83)    &652.0 (0.03)    &2510.0 (0.11)    &106 (0.00)    &637.0 (0.03)    &0 (0.00)    &22250


In [26]:
asrel_mapping ={
    4577: 20160301,
    5423: 20170201,
    6447: 20180401,
    7616: 20190801,
    8821: 20201001,
    9644: 20211001,
    10020: 20220501
}

asrel = pd.DataFrame()

for cycle in asrel_mapping.keys():
    _asrel = pd.read_csv(
        f"data/raw/asrel/{asrel_mapping[cycle]}.as-rel.txt.bz2",
        compression="bz2",
        sep="|",
        comment="#",
        names=["provider", "customer", "type"]
    )
    # asrel = asrel.append(_asrel)
    asrel = pd.concat([asrel, _asrel])
    


t1 = []
t2 = []
for cycle in [4577, 5423, 6447, 7616, 8821, 9644, 10020]:
    concat_df = pd.DataFrame()
    for _cycle in range(cycle -1, cycle + 2):
        # concat_df = concat_df.append(filtered_geoloc_hops[_cycle])
        concat_df = pd.concat([concat_df, filtered_geoloc_hops[_cycle]])

    G = nx.from_pandas_edgelist(
        concat_df.loc[concat_df["diff_rtt"] > 57],
        "near_node_id",
        "far_node_id",
    )

    nx.set_node_attributes(
        G,
        pd.Series(
            concat_df["near_node_asn"].values,
            index=concat_df["near_node_id"]
        ).to_dict(),
        'asn',
    )
    nx.set_node_attributes(
        G,
        pd.Series(
            concat_df["far_node_asn"].values,
            index=concat_df["far_node_id"]
        ).to_dict(),
        'asn',
    )


    nx.set_node_attributes(
        G,
        pd.Series(
            concat_df["flag_far_side_ixp"].values,
            index=concat_df["near_node_id"]
        ).to_dict(),
        'ixp_flag',
    )
    nx.set_node_attributes(
        G,
        pd.Series(
            concat_df["flag_near_side_ixp"].values,
            index=concat_df["far_node_id"]
        ).to_dict(),
        'ixp_flag',
    )

    t = len(G.edges())

    i = 0
    p2p = 0
    p2c = 0
    ixp = 0
    tor_uknown = 0
    unmapped = 0

    for node in G.nodes():
        for neighbor_id in G.neighbors(node):

            near_asn = int(G.nodes[node]["asn"])
            far_asn = int(G.nodes[neighbor_id]["asn"])

            if (near_asn != 0) and (near_asn != 0):
                if near_asn == far_asn:
                    i += 1
                else:
                    tor = asrel.loc[
                        ((asrel["provider"] == near_asn)
                         & (asrel["customer"] == far_asn))
                        | ((asrel["customer"] == near_asn)
                           & (asrel["provider"] == far_asn))
                    ]["type"].values

                    if len(tor) > 0:
                        tor = tor[0]
                    else:
                        tor = 1

                    if tor == -1:
                        p2c +=1
                    elif tor == 0:
                        p2p += 1
                    else:
                        if G.nodes[node]["ixp_flag"] or G.nodes[neighbor_id]["ixp_flag"]:
                            ixp +=1
                        else:
                            tor_uknown += 1
            else:
                unmapped += 1
    i /= 2
    p2p /= 2
    p2c /= 2
    tor_uknown /= 2
    
    
    t1.append((cycle, i, p2p + p2c, unmapped, i + p2p + p2c + unmapped))
    t2.append((cycle,  p2c, p2p, ixp, tor_uknown, t))


In [27]:
for cycle, intra, inter, unmapped, t in t1:
    print(f"{cycle}\
    &{int(intra)} ({(intra / float(t)):.02f})\
    &{int(inter)} ({(inter / float(t)):.02f})\
    &{int(unmapped)} ({(unmapped / float(t)):.02f})\
    &{int(t)}\\\\")

4577    &5101 (0.79)    &1354 (0.21)    &17 (0.00)    &6472\\
5423    &6173 (0.80)    &1538 (0.20)    &40 (0.01)    &7751\\
6447    &7086 (0.79)    &1888 (0.21)    &0 (0.00)    &8974\\
7616    &12406 (0.81)    &2746 (0.18)    &224 (0.01)    &15376\\
8821    &14632 (0.89)    &1750 (0.11)    &2 (0.00)    &16384\\
9644    &20677 (0.84)    &3876 (0.16)    &2 (0.00)    &24555\\
10020    &18398 (0.85)    &3162 (0.15)    &0 (0.00)    &21560\\


In [28]:
for cycle,  p2c, p2p,  ixp, tor_uknown, t in t2:
    print(f"{cycle}\
    &{int(p2c)} ({(p2c / float(t)):.02f})\
    &{int(p2p)} ({(p2p / float(t)):.02f})\
    &{int(ixp)} ({(ixp / float(t)):.02f})\
    &{int(tor_uknown)} ({(tor_uknown / float(t)):.02f})\\\\")

4577    &893 (0.13)    &461 (0.07)    &131 (0.02)    &197 (0.03)\\
5423    &1044 (0.13)    &494 (0.06)    &130 (0.02)    &310 (0.04)\\
6447    &1286 (0.14)    &602 (0.06)    &170 (0.02)    &211 (0.02)\\
7616    &1873 (0.12)    &873 (0.06)    &187 (0.01)    &403 (0.03)\\
8821    &1285 (0.08)    &465 (0.03)    &90 (0.01)    &212 (0.01)\\
9644    &2961 (0.12)    &915 (0.04)    &100 (0.00)    &548 (0.02)\\
10020    &2510 (0.11)    &652 (0.03)    &106 (0.00)    &637 (0.03)\\
